In [27]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import dash_table as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
from jupyter_dash import JupyterDash
from pandas_datareader import wb
import json	

stock_ticker = pd.read_excel(r"C:\Users\i5 lenov\Desktop\swevenaifin_complete\swevenaifin\datasets\allassets.xlsx")
stock_ticker = stock_ticker.set_index("Symbol")
options = []
for tic in stock_ticker.index:
    options.append({"label":"{} {}".format(tic,stock_ticker.loc[tic]["Name"]), "value":tic})
get_bootstrap_css = "https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta2/dist/css/bootstrap.min.css"
get_bootsrap_icon = "https://cdn.jsdelivr.net/npm/bootstrap-icons@1.4.0/font/bootstrap-icons.css"
app = JupyterDash(external_stylesheets=[get_bootstrap_css, get_bootsrap_icon],
                  meta_tags=[{"name" : "viewport",
                              "charset" :"utf-8",
                              "content" : "width=device-width,initial-scale=1, shrink-to-fit=no"}])
alerts_notif = dbc.Row(
    [
        dbc.Col(
            [dbc.NavLink(className="bi bi-bell",
                         href = "/page-2/1",
                        style = {"font-size" : "20px", "color" : "gray"}),],
            width = "auto"
        ),
    ],
    no_gutters = True,
    className = "ml-auto flex-nowrap mt-3 mt-md-0",
    align = "right",
    id = "responsivemenu-3-collapse"
)
messages = dbc.Row(
    [
        dbc.Col(
            dbc.NavLink(className = "bi bi-envelope",
                        href = "/page-3/1",
                        style = {"font-size" : "20px", "color" : "gray"}),
            width="auto"
        ),
    ],
    no_gutters = True,
    className = "ml-auto flex-nowrap mt-3 mt-md-0",
    align = "right",
     style = {"font-size" : "16px"},
    id = "responsivemenu-4-collapse"
)
profile = dbc.DropdownMenu(
    children=[
        dbc.DropdownMenuItem("Edit Profile",
                             href = "/page-4/1"),
        dbc.DropdownMenuItem("Privacy & Safety",
                             href = "/page-4/2"),
        dbc.DropdownMenuItem("Account Settings",
                             href = "/page-4/3"),
        dbc.DropdownMenuItem("Billing",
                             href = "/page-4/4"),
        dbc.DropdownMenuItem(divider = True),
        dbc.DropdownMenuItem("Sign Out",
                             href = "/page-4/5"),
    ],
    nav = True,
    in_navbar = True,
    className = "bi bi-person",
    direction = "left",
    style = {"font-size" : "20px"},
    id = "responsivemenu-5-collapse"
)
navbar = dbc.Navbar(
    [
        dbc.Col([],width=2),
        dbc.Col([
            dbc.Button(id = "toggle-button",
                       n_clicks = 0,
                       children = "",
                       outline = True,
                       className = "navbar-toggler-icon")],
            width = 1),
        dbc.Col([],
                width = 3),
        dbc.Col([messages],
                width = "auto"),
        dbc.Col([alerts_notif],
                width = "auto"),
        dbc.Col([], width = 2),
        dbc.Col([profile],
                width = 2)],
    color = "white",
    style={"margin-right" : "0rem",
           "margin-top" : "0rem",
           "margin-bottom" : "0.5rem",
           "padding" : "1rem 0rem"}) 
RESPONSIVE_MENU_STYLE = {
    "position" : "fixed",
    "top" : 0,
    "left" : 0,
    "bottom" : 0,
    "width" : "14rem",
    "height" : "100%",
    "margin-top" : "0rem",
    "margin-bottom" : "0rem",
    "z-index" : 1,
    "overflow-x" : "hidden",
    "transition" : "all 0.5s",
    "padding" : "0.5rem 1rem"
}
RESPONSIVE_MENU_HIDEN = {
    "position" : "fixed",
    "top" : 0,
    "left" : "-16rem",
    "bottom" : 0,
    "width" : "14rem",
    "height" : "100%",
    "z-index" : 1,
    "overflow-x" : "hidden",
    "transition" : "all 0.5s",
    "padding" : "0rem 0rem",
}
APP_CONTENT_STYLE = {
    "transition" : "margin-left .5s",
    "margin-left" : "14.5rem",
    "margin-right" : "0.5rem",
    "margin-bottom" : "0.5rem",
    "padding" : "0rem 0rem"
}
APP_CONTENT_STYLE1 = {
    "transition" : "margin-left .5s",
    "margin-left" : "0.5rem",
    "margin-right" : "0.5rem",
    "margin-bottom" : "0.5rem",
    "padding" : "0rem 0rem"
}
RESPONSIVE_RESPONSIVEMENU_1 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col(dbc.NavLink("Forecast Indicators",
                        href = "/page-1/1",
                        style = {"color" : "#616161"})),
            ],
            className = "my-1",
        ),
        style = {"cursor" : "pointer"},
        id = "responsivemenu-1",
    )
]

RESPONSIVE_SIDE_NAVIGATION_BAR = html.Div(
    [
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4("WorldViewer",
                                className = "btn btn-outline-primary"),
                        html.Hr(),
                        html.P(
                            "",
                            className = "lead"),
                        dbc.Nav(
                            RESPONSIVE_RESPONSIVEMENU_1,
                            vertical = True)
                    ]
                )
            ],
            id = "responsivesidebar")
    ]
)
INPUT_CARD = dbc.Card([
    dbc.CardBody([
        dbc.Row([
            dbc.Col([
                dbc.FormGroup([
                    dbc.Label("Select independent variables (over 3 stocks)"),
                    dcc.Dropdown(id = "independent-variable-input",
                                 className = "border-bottom",
                                 options = options,
                                 multi = True,
                                 value = [],
                                 placeholder = "Search stocks (independent variables)")])],
                width = 6),
            dbc.Col([
                dbc.FormGroup([
                    dbc.Label("Select the dependent variable"),
                    dcc.Dropdown(id = "dependent-variable-input",
                                 className = "border-bottom",
                                 options = options,
                                 multi = False,
                                 placeholder = "Search stock (dependent variable)")])],
                width = 6)]),     
        html.Br(),
        dbc.Row([
            dbc.Col([], width=5),
            dbc.Col([
                dbc.Button("Show results",
                           id = "worldviewer-submit-button",
                           color = "primary")],
                width = 4)])])])

OUTPUT_CARD = html.Div([dbc.Card([
    dbc.CardBody([
            html.H4("Statistical summary",
                    className="card-title",
                    style={"text-align": "center"})]),],
        style={"width": "auto","background-color": "#FAFAFA"}),
    html.Br(),
    dbc.Row([dbc.Col([html.Div(id="descriptive-statistics")])]),
    html.Br(),
    
    dbc.Card([
        dbc.CardBody([
            html.H4("Correlation",
                    className="card-title",
                    style={"text-align": "center"})]),],
        style={"width": "auto","background-color": "#FAFAFA"},),
    html.Br(),
    dbc.Row([dbc.Col([html.Div(id="pearson-correlation")])]),
    html.Br(),
    dbc.Card([
        dbc.CardBody([
            html.H4("Prediction",
                    className="card-title",
                    style={"text-align": "center"})]),],
        style={"width": "auto","background-color": "#FAFAFA"},),
    html.Br(),
    dbc.Row([dbc.Col([html.Div(id="linear-model-predictions")])]),
    html.Br(),
    dbc.Card([
        dbc.CardBody([
            html.H4("Coefficients and intercept",
                    className="card-title",
                    style={"text-align": "center"})]),],
        style={"width": "auto","background-color": "#FAFAFA"},),
    html.Br(),
    dbc.Row([dbc.Col([html.Div(id="linear-model-intercept-and-coefficients")])]),
    html.Br(),
    dbc.Card([
        dbc.CardBody([
            html.H4("Model evaluation",
                    className="card-title",
                    style={"text-align": "center"})]),],
        style={"width": "auto","background-color": "#FAFAFA"},),
    html.Br(),
    dbc.Row([dbc.Col([html.Div(id="linear-model-evaluation")])]),
    ])

WORDVIEWER_SUMMARY = html.Div([
    dbc.Row([dbc.Col([OUTPUT_CARD])])],
    id="collapse-worldviewer-page")
WORDVIEWER_LAYOUT = html.Div([
    html.Br(),
    dbc.Row([
        dbc.Col([INPUT_CARD],width=12)]),
    html.Br(),
    dbc.Row([
        dbc.Col([],width=5),
        dbc.Col([])]),
    dbc.Collapse([WORDVIEWER_SUMMARY],
                 id="collapse-worldviewer-menu")])
content = html.Div(
    id = "app-content",
    style=APP_CONTENT_STYLE)
app.layout = html.Div(
    [
        dcc.Store(id = "responsive-sidebar-click"),
        dcc.Store(id='independent-variables-values'),
        dcc.Store(id='dependent-variable-value'),       
        dcc.Location(id = "url"),
        navbar,
        RESPONSIVE_SIDE_NAVIGATION_BAR,
        content
    ]
)
@app.callback(
    [
        Output("responsivesidebar", "style"),
        Output("app-content", "style"),
        Output("responsive-sidebar-click", "data"),
    ],

    [Input("toggle-button", "n_clicks")],
    [
        State("responsive-sidebar-click", "data"),
    ]
)
def toggle_responsivesidebar(n, nclick):
    if n:
        if nclick == "SHOW":
            RESPONSIVE_MENU_style = RESPONSIVE_MENU_HIDEN
            APP_CONTENT_style = APP_CONTENT_STYLE1
            NO_OF_CURRENT_CLICKS = "HIDDEN"
        else:
            RESPONSIVE_MENU_style = RESPONSIVE_MENU_STYLE
            APP_CONTENT_style = APP_CONTENT_STYLE
            NO_OF_CURRENT_CLICKS = "SHOW"
    else:
        RESPONSIVE_MENU_style = RESPONSIVE_MENU_STYLE
        APP_CONTENT_style = APP_CONTENT_STYLE
        NO_OF_CURRENT_CLICKS = "SHOW"
    return RESPONSIVE_MENU_style, APP_CONTENT_style, NO_OF_CURRENT_CLICKS
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open
def set_navitem_class(is_open):
    if is_open:
        return "open"
    return ""
for i in [1, 2, 3, 4, 5]:
    app.callback(
        Output(f"responsivemenu-{i}-collapse", "is_open"),
        [Input(f"responsivemenu-{i}", "n_clicks")],
        [State(f"responsivemenu-{i}-collapse", "is_open")],
    )(toggle_collapse)
    app.callback(
        Output(f"responsivemenu-{i}", "className"),
        [Input(f"responsivemenu-{i}-collapse", "is_open")],
    )(set_navitem_class)
@app.callback(Output("app-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname in ["/", "/page-1/1"]:
        return  WORDVIEWER_LAYOUT
    elif pathname == "/page-1/2":
        return html.P("Worldview search")
    elif pathname == "/page-2/1":
        return html.P("Forecast Indicators")
    elif pathname == "/page-3/1":
        return html.P("Message")
    elif pathname == "/page-4/1":
        return html.P("Profile")
    elif pathname == "/page-5/1":
        return html.P("Edit Profile")
    elif pathname == "/page-5/2":
        return html.P("Privacy & Safety")
    elif pathname == "/page-5/3":
        return html.P("Account Settings")
    elif pathname == "/page-5/4":
        return html.P("Billing")
    elif pathname == "/page-5/5":
        return html.P("Sign Out")
    return dbc.Jumbotron(
        [
            html.H1("Page not found",
                    className = "text-danger"),
            html.Hr(),
            html.P(f"{pathname} not found."),
        ]
    ) 

@app.callback(Output("collapse-worldviewer-menu", "is_open"),
              [Input("worldviewer-submit-button", "n_clicks")])
def toggle_collapse_worldviewer_menu(n):
    return n
@app.callback(Output("independent-variables-values", "data"),
              [Input("independent-variable-input", "value")])
def independepnt_variables_dataset(stock_ticker):
    start = "2018-01-01"
    end = "2021-09-01"
    df = web.get_data_yahoo(stock_ticker, start = start, end = end)
    new1 = pd.DataFrame(df.Close)
    new1 = new1.dropna()
    return new1.to_json(date_format = "iso", orient = "split")

@app.callback(Output("dependent-variable-value", "data"),
              [Input("dependent-variable-input", "value")])
def dependent_variable_dataset(stock_ticker):
    start = "2018-01-01"
    end = "2021-09-01"
    trace = []
    for tic in stock_ticker:
        df = web.get_data_yahoo(stock_ticker, start = start, end = end)
        new1 = pd.DataFrame(df.Close)
        new1 = new1.dropna()
        labels = stock_ticker
        new1.columns = [labels]
        return new1.to_json(date_format = "iso", orient = "split")

    
@app.callback(Output("descriptive-statistics", "children"),
              [Input("independent-variables-values", "data")],
              [State("dependent-variable-value", "data")])
def descriptive_statistics(independent_variable, dependent_variable):
    ind_variable = pd.read_json(independent_variable, orient = "split")
    dep_variable = pd.read_json(dependent_variable, orient = "split")
    ind_dep_variable = pd.concat([ind_variable, dep_variable], axis = 1)
    descritpive_statistics = ind_dep_variable.describe()
    descritpive_statistics = descritpive_statistics.reset_index()
    data = descritpive_statistics.to_dict("rows")
    columns = [{"name": i, "id": i,} for i in (descritpive_statistics.columns)]
    return dt.DataTable(data = data, columns = columns, style_table = {"overflow": "auto",
                                                                       "striped":"True",
                                                                       "bordered":"True",
                                                                       "hover":"True"})
@app.callback(Output("pearson-correlation", "children"),
              [Input("independent-variables-values", "data")],
              [State("dependent-variable-value", "data")])
def pearson_correlation(independent_variable, dependent_variable):
    ind_variable = pd.read_json(independent_variable, orient = "split")
    dep_variable = pd.read_json(dependent_variable, orient = "split")
    ind_dep_variable = pd.concat([ind_variable, dep_variable], axis = 1)
    ind_dep_variable = ind_dep_variable.dropna()
    pearson_correlation = ind_dep_variable.corr()
    pearson_correlation = pearson_correlation.reset_index()
    data = pearson_correlation.to_dict("rows")
    columns = [{"name": i, "id": i,} for i in (pearson_correlation.columns)]
    return dt.DataTable(data = data, columns = columns, style_table = {"overflow": "auto",
                                                                       "striped":"True",
                                                                       "bordered":"True",
                                                                       "hover":"True"})
@app.callback(Output("linear-model-predictions", "children"),
              [Input("independent-variables-values", "data")],
              [State("dependent-variable-value", "data")])
def linear_model_predictions(independent_variable, dependent_variable):
    ind_variable = pd.read_json(independent_variable, orient = "split")
    dep_variable = pd.read_json(dependent_variable, orient = "split")
    ind_dep_variables = pd.concat([ind_variable, dep_variable], axis = 1)
    x = ind_dep_variables.iloc[:,0:-1].values
    y = ind_dep_variables.iloc[::,-1].values
    y = np.array(y)
    x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2, shuffle = False)
    linear_model = LinearRegression(n_jobs = -1)
    linear_model.fit(x_train, y_train)
    linear_model_predictions = pd.DataFrame(linear_model.predict(x_test), columns = ["Forecast"])
    linear_model_predictions = linear_model_predictions.head(20)
    linear_model_predictions = linear_model_predictions.reset_index()
    data = linear_model_predictions.to_dict("rows")
    columns = [{"name": i, "id": i,} for i in (linear_model_predictions.columns)]
    return dt.DataTable(data = data, columns = columns, style_table = {"overflow": "auto",
                                                                       "striped":"True",
                                                                       "bordered":"True",
                                                                       "hover":"True"})

@app.callback(Output("linear-model-intercept-and-coefficients", "children"),
              [Input("independent-variables-values", "data")],
              [State("dependent-variable-value", "data")])
def linear_model_intercept_and_coefficients(independent_variable, dependent_variable):
    ind_variable = pd.read_json(independent_variable, orient = "split")
    dep_variable = pd.read_json(dependent_variable, orient = "split")
    ind_dep_variables = pd.concat([ind_variable, dep_variable], axis = 1)
    x = ind_dep_variables.iloc[:,0: -1].values
    y = ind_dep_variables.iloc[::, -1].values
    y = np.array(y)
    x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2, shuffle = False)
    linear_model = LinearRegression(n_jobs = -1)
    linear_model.fit(x_train, y_train)
    linear_model_predictions = linear_model.predict(x_test)
    labels = str("Coef_") + ind_variable.columns 
    intercept = pd.DataFrame(pd.Series(linear_model.intercept_))
    intercept.columns = ["Intercept"]
    coefficients = pd.DataFrame(linear_model.coef_).transpose()
    coefficients.columns = labels
    intercept_coefficients = pd.concat([intercept, coefficients],axis=1).transpose()
    intercept_coefficients.columns = ["Values"]
    intercept_coefficients = intercept_coefficients.reset_index()
    data = intercept_coefficients.to_dict("rows")
    columns = [{"name": i, "id": i,} for i in (intercept_coefficients.columns)]
    return dt.DataTable(data = data, columns = columns, style_table = {"overflow": "auto",
                                                                       "striped":"True",
                                                                       "bordered":"True",
                                                                       "hover":"True"})
@app.callback(Output("linear-model-evaluation", "children"),
              [Input("independent-variables-values", "data")],
              [State("dependent-variable-value", "data")])
def linear_model_evaluation(independent_variable, dependent_variable):
    ind_variable = pd.read_json(independent_variable, orient = "split")
    dep_variable = pd.read_json(dependent_variable, orient = "split")
    ind_dep_variables = pd.concat([ind_variable, dep_variable], axis = 1)
    date = ind_dep_variables.index
    x = ind_dep_variables.iloc[:,0: -1].values
    y = ind_dep_variables.iloc[::, -1].values
    y = np.array(y)
    x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2, shuffle = False)
    linear_model = LinearRegression(n_jobs = -1)
    linear_model.fit(x_train, y_train)
    linear_model_predictions = linear_model.predict(x_test)
    mean_absolute_error = metrics.mean_absolute_error(y_test, linear_model_predictions)
    mean_squared_error = metrics.mean_squared_error(y_test, linear_model_predictions)
    root_mean_squared_error = np.sqrt(MSE)
    R2 = metrics.r2_score(y_test, linear_model_predictions)
    explained_variance_score = metrics.explained_variance_score(y_test, linear_model_predictions)
    mean_gamma_deviance = metrics.mean_gamma_deviance(y_test, linear_model_predictions)
    mean_poisson_deviance = metrics.mean_poisson_deviance(y_test, linear_model_predictions)
    linear_model_evaluation = [[mean_absolute_error, mean_squared_error, root_mean_squared_error,
                                R2, explained_variance_score, mean_gamma_deviance, mean_poisson_deviance]]
    linear_model_evaluation_data = pd.DataFrame(linear_model_evaluation,
                                         index = ["Values"],
                                         columns = ["Mean absolute error", "Mean squared error", "Root mean squared error",
                                                    "R2", "Explained variance score", "Mean gamma deviance", "Mean Poisson deviance"])
    data = linear_model_evaluation_data.to_dict("rows")
    columns = [{"name": i, "id": i,} for i in (linear_model_evaluation_data.columns)]
    return dt.DataTable(data = data, columns = columns, style_table = {"overflow": "auto",
                                                                       "striped":"True",
                                                                       "bordered":"True",
                                                                       "hover":"True"})

app.run_server(mode = "external",
               dev_tools_ui = False,
               dev_tools_props_check = False)



Dash app running on http://127.0.0.1:8050/
